In [1]:
class CFG:
    exp = '005'
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + CFG.exp
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
CPU times: user 35.7 ms, sys: 27.3 ms, total: 63 ms
Wall time: 1.47 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [110]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{CFG.exp}')
output_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/stacking{CFG.exp}')
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [6]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
print(get_score(oof_df['score'], oof_df['pred']))
oof_df.head()

0.8303980702351716


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843


In [8]:
tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')

In [9]:
train = oof_df.copy()

# 特徴量エンジニアリング
def tok_len(sentence):
    return len(tokenizer.tokenize(sentence))

def feature_engineering(df):
    df['section'] = df['context'].map(lambda s: s[0])

    df['anchor_tok_len'] = df['anchor'].map(tok_len)
    df['target_tok_len'] = df['target'].map(tok_len)
    df['context_tok_len'] = df['context_text'].map(tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']


train = feature_engineering(train)
train.head()

,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10


In [12]:
# カテゴリ変数の変換
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [16]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'pred', 'section', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded'], dtype='object')


In [17]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['pred', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded']


In [98]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [100]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [101]:
best_params = tuning_params(train, params)
best_params

Output hidden; open in https://colab.research.google.com to view.

In [95]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [102]:
pred = train_fn(train, best_params)

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0283152	valid_1's l2: 0.0275488
[200]	training's l2: 0.0216391	valid_1's l2: 0.0209347
[300]	training's l2: 0.0204888	valid_1's l2: 0.0198113
[400]	training's l2: 0.020212	valid_1's l2: 0.0196023
[500]	training's l2: 0.020094	valid_1's l2: 0.0195742
Early stopping, best iteration is:
[496]	training's l2: 0.0200972	valid_1's l2: 0.0195739
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0279099	valid_1's l2: 0.0281978
[200]	training's l2: 0.0211601	valid_1's l2: 0.0218634
[300]	training's l2: 0.0200157	valid_1's l2: 0.0209705
[400]	training's l2: 0.0197449	valid_1's l2: 0.0208793
[500]	training's l2: 0.0196237	valid_1's l2: 0.020895
Early stopping, best iteration is:
[410]	training's l2: 0.0197296	valid_1's l2: 0.0208759
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0282579	valid_1's l2: 0.0284706
[200]	training's l2: 0.0216772	v

In [120]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

,feature,importance
0,pred,36936.541034
2,target_tok_len,253.948249
11,section_encoded,191.311695
5,len_anc_tgt_diff,70.419601
1,anchor_tok_len,58.263946
6,len_anc_tgt_div,55.948327
10,len_tgt_cnt_div,47.912607
8,len_anc_cnt_div,35.997038
4,input_len,34.743194
7,len_anc_cnt_diff,19.366490


In [103]:
# optunaのbest params
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8334158647629792


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.410609
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.568814
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.207365
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.416627
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.048620


In [78]:
# num_leaves: 3
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8333142080336173


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.407946
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.559879
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.209868
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.409758
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.050144


In [74]:
# min_data_in_leaf: 100
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8330000762684592


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.415017
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.569874
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.193149
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.417659
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.047891


In [70]:
# max_depth: 5
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8330834975153025


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.423242
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.591858
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.187980
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.413583
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.046981


In [69]:
# max_depth: 7
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8330834975153025


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.423242
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.591858
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.187980
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.413583
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.046981


In [61]:
# num_leaves: 20
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8329134065443831


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.428995
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.581654
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.189729
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.408904
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.054688


In [57]:
# num_leaves: 60
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8313463268525216


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.413619
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.624658
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.182667
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.382782
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.047719


In [53]:
# num_leaves: 80
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8308542050164229


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.443496
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.618952
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.197005
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.384551
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.041946


In [49]:
# dart
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8321936374420458


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.361724
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.504333
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.159029
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.338017
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.047352


In [45]:
# gbdt
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8303980702351716
0.8325060723617361


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.411608,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.437311
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.595921,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.588248
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.168890,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.183208
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.438424,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.408232
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000843,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.047689


In [113]:
ColabConfig.dataset_new = False
ColabConfig.dataset_note = '"colsを保存"'

In [114]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()

Starting upload for file section_encoder.pkl
100% 323/323 [00:00<00:00, 751B/s]
Upload successful: section_encoder.pkl (323B)
Starting upload for file lgb_fold0.txt
100% 564k/564k [00:00<00:00, 1.36MB/s]
Upload successful: lgb_fold0.txt (564KB)
Starting upload for file lgb_fold1.txt
100% 468k/468k [00:00<00:00, 1.03MB/s]
Upload successful: lgb_fold1.txt (468KB)
Starting upload for file lgb_fold2.txt
100% 676k/676k [00:00<00:00, 1.74MB/s]
Upload successful: lgb_fold2.txt (676KB)
Starting upload for file lgb_fold3.txt
100% 428k/428k [00:00<00:00, 1.04MB/s]
Upload successful: lgb_fold3.txt (428KB)
Starting upload for file cols.pkl
100% 256/256 [00:00<00:00, 635B/s]
Upload successful: cols.pkl (256B)
Dataset version is being created. Please check progress at https://www.kaggle.com/hanejiyuto/PPPM-stacking-exp005
